In [2]:
import os
from dotenv import load_dotenv
import weaviate

load_dotenv()

weaviate_apikey = os.getenv("WEAVIATE_APIKEY")
client = weaviate.Client(
    "http://localhost:8080",
    auth_client_secret=weaviate.auth.AuthApiKey(weaviate_apikey),
)

# How many paragraphs we have in the database?
client.query.aggregate("passage").with_meta_count().do()


{'data': {'Aggregate': {'Passage': [{'meta': {'count': 64}}]}}}

In [3]:
# Ask a sample question
question = {"concepts": ["What is Antibacterial Activity?"]}

result = (
    client.query.get("Passage", ["text_content"])
    .with_near_text(question)
    .with_limit(10)
    .with_additional(["vector", "distance", "id"])
    .do()
)


In [4]:
# What are the related paragraphs?
for x in result["data"]["Get"]["Passage"]:
    print(x["text_content"], "\n", "-" * 180)

of salivation, it is not surprising that a large number of pharmacologic agents affect salivary flow and possibly composition as well. A number of agents cause increased salivation (e.g., urecholine, neostigmine, physostigmine, potassium iodide), but the vast majority of medications affecting salivary glands result in decreased flow. More than 250 drugs include dry mouth as a possible side effect. The most widely used are antidepressants (e.g., amitryptyline), anticholinergics, antispasmodics (e.g., propantheline), an-
50 MANDEL

tihypertensives (e.g., clondine), antipsychotics (e.g., thorazine) and bronchodilators (e.g., ephedrine).
Extended use of medications that reduce salivary flow not only results in discomfort, but may also alter taste perception and the ability to form an adequate bolus; this is a combination that can impair adequate food intake. In addition, the loss of the protective components of saliva can result in a rapid, marked increase in caries. Where discontinuing me

In [7]:
raw_vec = result["data"]["Get"]["Passage"][0]["_additional"]["vector"]
raw_vec

[0.07359388,
 0.053982597,
 -0.0029394866,
 -0.0036199677,
 0.007431128,
 -0.014394393,
 0.078719534,
 -0.00070695573,
 0.024440967,
 -0.075287364,
 0.04103957,
 0.06419351,
 -0.007922345,
 0.0075776866,
 -0.0151453465,
 0.02868523,
 0.10931299,
 0.08288019,
 0.042927045,
 -0.032088626,
 -0.047838654,
 0.00024898924,
 0.0377532,
 -0.005189077,
 0.053609665,
 -0.0039797155,
 -0.01616472,
 0.016708463,
 -0.0661763,
 -0.0355448,
 -0.011292636,
 0.080904715,
 0.021070499,
 -0.06432104,
 -0.04052161,
 0.047185335,
 -0.004343012,
 -0.05390706,
 -0.074601285,
 -0.044389553,
 -0.09365932,
 0.023749415,
 -0.056144245,
 0.011799675,
 -0.031120094,
 -0.036501646,
 -0.17017704,
 0.030825682,
 -0.063720316,
 -0.0061762487,
 -0.002248286,
 -0.0061907084,
 0.02447243,
 0.011340793,
 0.010378351,
 0.02618676,
 -0.09938893,
 0.042872135,
 -0.021685634,
 -0.015213139,
 0.027922507,
 0.021403749,
 -0.007029657,
 0.047127713,
 0.035867933,
 0.030696455,
 -0.028139453,
 -0.04077129,
 -0.0060783653,
 0.0237